# 🚀 AI Multi-Agent Codelab
<a href="https://colab.research.google.com/github/chitoan1992/devfest-multiagent/blob/main/codelab_multiagent_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
## Build Your AI Startup Team in 30 Minutes!



In this codelab, you will learn:
- Create an AI Agent that can plan
- Give your Agent the ability to use tools (search, image generation)
- Build a Multi-Agent system with an Orchestrator
- Deploy a landing page for your startup idea

---

### 🛠️ Tech Stack

| Tool | Purpose |
|------|----------|
| DeepAgents (LangChain) | Agent orchestration |
| Gemini 2.0 Flash | LLM backbone |
| Gemini Imagen 3 | Image generation |
| Google AI Studio | Landing page canvas |
| Vercel/Netlify | 1-click deployment |

![AI Multi-Agent Workflow](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/full_workflow.png)

---
## 📦 Cell 0: Installation & Setup Images

> 💡 **Instructions:** Upload the `images/` folder to Colab to display illustration images

In [33]:
!pip install deepagents langchain-google-genai google-genai nest_asyncio rich -q

---
## 🔑 Cell 1: Setup & API Key

In [34]:
import os
from google.colab import userdata

# Get API key from Colab Secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("✅ API Key configured!")

✅ API Key configured!


---
## 🔧 Cell 2: Imports & Client Setup

In [35]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types
import asyncio
import nest_asyncio
from IPython.display import display, clear_output, HTML
from rich.console import Console
from rich.panel import Panel

# Enable nested async for Colab
nest_asyncio.apply()

# Gemini model for DeepAgents
model = init_chat_model("google_genai:gemini-2.0-flash")

# Gemini client for Imagen
gemini_client = genai.Client()

# Rich console
console = Console(force_terminal=True, force_interactive=True)

print("✅ Setup complete!")

✅ Setup complete!


In [36]:
"""Utility functions for displaying messages and prompts in Jupyter notebooks."""

import json

from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()


def format_message_content(message):
    """Convert message content to displayable string."""
    parts = []
    tool_calls_processed = False

    # Handle main content
    if isinstance(message.content, str):
        parts.append(message.content)
    elif isinstance(message.content, list):
        # Handle complex content like tool calls (Anthropic format)
        for item in message.content:
            if item.get("type") == "text":
                parts.append(item["text"])
            elif item.get("type") == "tool_use":
                parts.append(f"\n🔧 Tool Call: {item['name']}")
                parts.append(f"   Args: {json.dumps(item['input'], indent=2, ensure_ascii=False)}")
                parts.append(f"   ID: {item.get('id', 'N/A')}")
                tool_calls_processed = True
    else:
        parts.append(str(message.content))

    # Handle tool calls attached to the message (OpenAI format) - only if not already processed
    if (
        not tool_calls_processed
        and hasattr(message, "tool_calls")
        and message.tool_calls
    ):
        for tool_call in message.tool_calls:
            parts.append(f"\n🔧 Tool Call: {tool_call['name']}")
            parts.append(f"   Args: {json.dumps(tool_call['args'], indent=2, ensure_ascii=False)}")
            parts.append(f"   ID: {tool_call['id']}")

    return "\n".join(parts)


def format_messages(messages):
    """Format and display a list of messages with Rich formatting."""
    for m in messages:
        msg_type = m.__class__.__name__.replace("Message", "")
        content = format_message_content(m)

        if msg_type == "Human":
            console.print(Panel(content, title="🧑 Human", border_style="blue"))
        elif msg_type == "Ai":
            console.print(Panel(content, title="🤖 Assistant", border_style="green"))
        elif msg_type == "Tool":
            console.print(Panel(content, title="🔧 Tool Output", border_style="yellow"))
        else:
            console.print(Panel(content, title=f"📝 {msg_type}", border_style="white"))


def format_message(messages):
    """Alias for format_messages for backward compatibility."""
    return format_messages(messages)


def show_prompt(prompt_text: str, title: str = "Prompt", border_style: str = "blue"):
    """Display a prompt with rich formatting and XML tag highlighting.

    Args:
        prompt_text: The prompt string to display
        title: Title for the panel (default: "Prompt")
        border_style: Border color style (default: "blue")
    """
    # Create a formatted display of the prompt
    formatted_text = Text(prompt_text)
    formatted_text.highlight_regex(r"<[^>]+>", style="bold blue")  # Highlight XML tags
    formatted_text.highlight_regex(
        r"##[^#\n]+", style="bold magenta"
    )  # Highlight headers
    formatted_text.highlight_regex(
        r"###[^#\n]+", style="bold cyan"
    )  # Highlight sub-headers

    # Display in a panel for better presentation
    console.print(
        Panel(
            formatted_text,
            title=f"[bold green]{title}[/bold green]",
            border_style=border_style,
            padding=(1, 2),
        )
    )

---
# 📍 MILESTONE 1: First Agent + Planning (10 min)



**Goal:** Create your first agent with planning capabilities

> 💡 An Agent doesn't just answer questions. An Agent **THINKS** and **PLANS**.

![Milestone 1: Planning Agent](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone1_planning_agent.png)

In [37]:
# Planning Agent with DeepAgents
planning_agent = create_deep_agent(
    model=model,
    system_prompt="""
    You are a Planning Agent. When receiving a task:
    1. Break down the task into smaller steps
    2. Reply concisely and to the point
    """
)

# Run agent with streaming
result = planning_agent.invoke({"messages":
    [{"role": "user", "content": "Plan to build a landing page for an AI startup"}]})
format_messages(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Plan to build a landing page for an AI startup                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "Define the target audience and goals of the landing page.",                                   │
│       "status": "in_progress"                                                                                   │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Research competitor landing pages for inspiration and best practices.",                       │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Create a wireframe and design mockup of the landing page.",                                   │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Write compelling copy that highlights the AI startup's value proposition.",                   │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Develop the landing page using HTML, CSS, and JavaScript.",                                   │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Optimize the landing page for search engines (SEO).",                                         │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Test the landing page on different devices and browsers.",                                    │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                   

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': 'Define the target audience and goals of the landing page.', 'status':        │
│ 'in_progress'}, {'content': 'Research competitor landing pages for inspiration and best practices.', 'status':  │
│ 'pending'}, {'content': 'Create a wireframe and design mockup of the landing page.', 'status': 'pending'},      │
│ {'content': "Write compelling copy that highlights the AI startup's value proposition.", 'status': 'pending'},  │
│ {'content': 'Develop the landing page using HTML, CSS, and JavaScript.', 'status': 'pending'}, {'content':      │
│ 'Optimize the landing page for search engines (SEO).', 'status': 'pending'}, {'content': 'Test the landing page │
│ on different devices and browsers.', 'status': 'pending'}, {'content': 'Deploy the landing page to a web        │
│ server.', 'status': 'pending'}, {'content': "Track the landing page's performance and make improvements.",      │
│ 'status': 'pending'}]                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Okay, I've created a to-do list to guide the process of building the landing page. I've marked the first step,  │
│ "Define the target audience and goals of the landing page," as in progress.                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### ✅ Checkpoint 1
**Can you see the plan?** Raise your hand if yes!

---

# 📍 MILESTONE 2: Agent + Tools (10 min)



**Goal:** Give your agent the ability to call tools - generate images with Imagen + search the web

> 💡 An Agent is smart but 'trapped' in text. Tools = giving the agent **HANDS AND FEET** to interact with the real world.

![Milestone 2: Agent with Tools](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone2_tools.png)

In [38]:
# Tool 1: Generate image with Imagen
@tool
def generate_image(prompt: str) -> str:
    """Generate a hero image from a text prompt using the Gemini Imagen API.

    Use this tool when you need to create images for landing pages, pitch decks,
    or any visual assets.

    Args:
        prompt: Detailed description of the image to create (English works best)

    Returns:
        Confirmation message with path to saved image
    """
    response = gemini_client.models.generate_images(
        model="imagen-4.0-ultra-generate-001",
        prompt=prompt,
        config={"number_of_images": 1}
    )
    image = response.generated_images[0]
    image.image.save("hero_image.png")
    return "✅ Created hero_image.png - Show it to the user!"


# Tool 2: Search web with Google Grounded Search (built-in Gemini)
@tool
def search_web(query: str) -> str:
    """Search the web to research competitors, market trends, or information.

    Uses Google Search built into the Gemini API (Grounded Search).

    Args:
        query: The search query

    Returns:
        Search results from Google
    """
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=query,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    return response.text


print("✅ Tools defined!")

✅ Tools defined!


In [ ]:
# Agent with custom tools
visual_agent = create_deep_agent(
    model=model,
    tools=[search_web, generate_image],
    system_prompt="""
    You are a Visual Agent. Your tasks:
    1. Use search_web to research information
    2. Use generate_image to create visual assets
    """
)

result = visual_agent.invoke({"messages":
    [{"role": "user", "content": "Search the web about Journey to the West and create an infographic illustration"}]})
format_messages(result["messages"])

In [ ]:
# Display the generated image
from IPython.display import Image, display
display(Image(filename='hero_image.png'))

### ✅ Checkpoint 2
**Did you generate an image?** Show us!

---

# 📍 MILESTONE 3: Personalization Agent (10 min)

**Goal:** Agent receives user profile and suggests suitable startup ideas

> 💡 The best AI is AI that understands **YOU**. Paste your LinkedIn/Facebook profile → Agent suggests ideas that match your skills.

![Milestone 3: Personalization Agent](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone3_personalization.png)

In [ ]:
# ============================================
# STEP 1: User enters profile
# ============================================
# Template for user input (or paste from LinkedIn/Facebook)

user_profile = {
    "name": "John Doe",
    "age": 22,
    "profession": "Frontend Developer",
    "interests": ["AI", "Startup", "Gaming"],
    "bio": """
    Passionate frontend developer with 2 years of experience
    in React and TypeScript. Love building beautiful UIs and exploring
    AI technologies. Active in local tech communities.
    """
}

print(f"Profile loaded for: {user_profile['name']}")

In [ ]:
# Personalization Agent
personalization_agent = create_deep_agent(
    model=model,
    system_prompt="""
    You are a Personal Branding Expert. Based on the user's profile:
    1. Analyze their strengths
    2. Suggest 3 startup ideas (ranked by fit)
    3. Create a personal tagline
    4. Suggest a pitch style
    Reply in English
    """
)

# Run agent with streaming
result = personalization_agent.invoke({"messages":
    [{"role": "user", "content": f"Analyze this profile and suggest startup ideas:\n{user_profile}"}],
})
format_messages(result["messages"])

In [ ]:
print(result["messages"][-1].content)

In [ ]:
# ============================================
# STEP 4: User chooses an idea
# ============================================

chosen_idea = input("Which idea do you choose? (1/2/3): ")
print(f"\n✅ You chose idea #{chosen_idea}!")
print("\n👉 Next: Build a Multi-Agent system for your startup!")

### ✅ Checkpoint 3
**Have you chosen your startup idea?**

---

# 📍 MILESTONE 4: Multi-Agent + Critic (20 min)



**Goal:** Build a team of agents with orchestration + a Critic to improve output

> 💡 One agent is good. But a **TEAM of agents = unstoppable**. And every team needs a critic for quality control.



![Milestone 4: Multi-Agent Orchestra](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone4_multiagent.png)

In [ ]:
# ============================================
# DEFINE SUB-AGENTS
# ============================================

# Sub-agent 1: Pitch Generator
pitch_subagent = {
    "name": "pitch-generator",
    "description": "Creates pitch content for startups. Call when you need to generate pitch deck content.",
    "system_prompt": """
    You are a Pitch Expert. Create pitches following the 4 PILLARS:

    1. EXTREME CLARITY - One sentence description, everyone understands
    2. OBVIOUS VALUE - "With it vs without it?" clearly answered
    3. LOW FRICTION - Can be demoed in 30 seconds
    4. SMART PACKAGING - Visual storytelling

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎯 ONE-LINER: [Max 10 words]

    😫 PROBLEM:
    [Describe the pain point in 2 sentences]

    ✨ SOLUTION:
    [Describe the solution in 2 sentences]

    🔑 KEY FEATURES:
    1. [Feature 1]
    2. [Feature 2]
    3. [Feature 3]

    📊 KEY METRIC: [1 specific number]

    🎬 CTA: [Call to action]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 2: Critic
critic_subagent = {
    "name": "critic",
    "description": "Evaluates and improves pitch quality. Call AFTER getting pitch content.",
    "system_prompt": """
    You are a strict Pitch Critic. Evaluate based on 4 criteria (1-5 points each):

    1. CLARITY (1-5): Does the listener understand in 10 seconds?
    2. VALUE (1-5): Is "with it vs without it" clear?
    3. UNIQUE (1-5): How is it different from existing solutions?
    4. VIBE (1-5): Would GenZ find it cool?

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 SCORE: X/20

    ✅ STRENGTHS:
    - [Point 1]
    - [Point 2]

    ⚠️ NEEDS IMPROVEMENT:
    - [Point 1]
    - [Point 2]

    💡 BETTER TAGLINE SUGGESTION: "..."

    🎯 VERDICT: [APPROVED ✓ if ≥16/20 | NEEDS IMPROVEMENT if <16/20]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    If score < 16: Explain SPECIFICALLY what needs improvement.
    """,
    "tools": []
}

# Sub-agent 3: Landing Page Content Builder (OUTPUT PROMPT FOR AI STUDIO)
landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Creates AI Studio prompt for landing page. User copies this prompt into AI Studio Canvas mode to generate HTML. Call AFTER pitch is approved.",
    "system_prompt": """
    You are a Landing Page Content Specialist. Your task is to create a PROMPT + CONTENT
    for the user to copy into Google AI Studio Canvas mode to generate HTML.

    Based on the approved pitch content, create output in this format:

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📋 COPY INTO AI STUDIO CANVAS MODE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Create a complete HTML landing page for a startup with this information:

    ## STARTUP INFO
    - Name: [Startup name from pitch]
    - Tagline: [One-liner from pitch]

    ## HERO SECTION
    - Headline: [Compelling headline from pitch]
    - Subheadline: [Short description of the solution]
    - CTA Button: [CTA button text from pitch]

    ## PROBLEM → SOLUTION
    - Problem: [Pain point from pitch]
    - Solution: [Solution from pitch]

    ## 3 KEY FEATURES (with emoji icons)
    1. 🎯 [Feature 1]: [Short description]
    2. ⚡ [Feature 2]: [Short description]
    3. 🔒 [Feature 3]: [Short description]

    ## STYLE REQUIREMENTS
    - Color scheme: Modern gradient Purple (#7C3AED) → Green (#10B981)
    - Font: Inter (Google Fonts)
    - Layout: Mobile-first responsive
    - Effects: Smooth hover animations, subtle shadows
    - Sections: Hero, Problem/Solution, Features, CTA, Footer

    ## FOOTER
    - Copyright text with current year
    - Social media placeholder links (Facebook, Twitter, LinkedIn)

    Create complete HTML that can be immediately deployed to Vercel/Netlify.

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📌 HOW TO USE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    1. Open aistudio.google.com
    2. Select Canvas mode (or create new prompt)
    3. Paste the entire prompt above (the part in the box)
    4. Click Generate
    5. Preview and download HTML
    6. Deploy to Vercel/Netlify
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 4: Social Post Writer
social_subagent = {
    "name": "social-writer",
    "description": "Creates Facebook post to share results. Call LAST after having landing page prompt.",
    "system_prompt": """
    Create a Facebook post for participants to share their results:

    REQUIREMENTS:
    - Tone: Excited but not cringe
    - Emoji: 3-5, placed appropriately
    - Max 280 characters
    - Required hashtags: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI

    TEMPLATE:
    🔥 [Hook - 1 attention-grabbing sentence]

    💡 Idea: [Startup name]
    🤖 [Number] AI agents working for me
    🌐 Live: [URL placeholder]

    [Hashtags]
    """,
    "tools": []
}

print("✅ Sub-agents defined!")

In [ ]:
# ============================================
# CREATE ORCHESTRATOR
# ============================================
orchestrator = create_deep_agent(
    model=model,
    subagents=[pitch_subagent, critic_subagent, landing_page_subagent, social_subagent],
    system_prompt="""
    You are the Startup Builder Orchestrator. Workflow:
    1. Call pitch-generator to create a pitch
    2. Call critic to evaluate (loop if < 16/20)
    3. Call landing-page-builder to create AI Studio prompt (NOT HTML directly)
    4. Call social-writer

    IMPORTANT NOTE:
    - landing-page-builder will output a PROMPT that user copies into Google AI Studio Canvas mode to generate HTML
    - Do NOT output HTML code directly
    """
)
print("✅ Orchestrator ready!")

In [ ]:
# ============================================
# RUN FULL PIPELINE
# ============================================
chosen_idea = "AI-powered study buddy for students"

result = orchestrator.invoke({"messages":
    [{"role": "user", "content": f"Build startup for idea: {chosen_idea}. Create pitch, get critic approval, HTML, Facebook post."}],
})

format_messages(result["messages"])

In [ ]:
print(result["messages"][-1].content)

### ✅ Checkpoint 4
**Has your pitch been approved by the Critic (≥16/20)?**

---

# 🎮 VIBE CODING GAME: AI Startup in 30 Minutes!



## Workflow



### Timeline

| Time | Task |
|------|------|
| 0-8 min | Personalization + Planning |
| 8-15 min | Pitch + Critic + Image |
| 15-25 min | AI Studio → HTML → Deploy |
| 25-30 min | Social post + Prepare pitch |

## 🚀 Full Starter Pack (Copy & Use!)

In [ ]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types

# Gemini model for DeepAgents
model = init_chat_model("google_genai:gemini-2.0-flash")

# Gemini client for Imagen
gemini_client = genai.Client()

# ============================================
# DEFINE ALL SUB-AGENTS
# ============================================

# Sub-agent 1: Personalization Agent
personalization_subagent = {
    "name": "personalization",
    "description": "Analyzes user profile and suggests suitable startup ideas. Call FIRST when user profile is provided.",
    "system_prompt": """
    You are a Personal Branding Expert. Based on the user's profile:

    1. Analyze strengths
    2. Suggest 3 startup ideas (ranked by profile fit)
    3. Create a personal tagline
    4. Suggest a suitable pitch style

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    💪 YOUR STRENGTHS:
    - [Strength 1]
    - [Strength 2]
    - [Strength 3]

    💡 3 SUITABLE STARTUP IDEAS:
    1. [Idea 1] - [Why it fits]
    2. [Idea 2] - [Why it fits]
    3. [Idea 3] - [Why it fits]

    🏷️ PERSONAL TAGLINE: "[Tagline]"

    🎤 SUGGESTED PITCH STYLE: [Style that fits personality]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Reply in English, concise and actionable.
    """,
    "tools": []
}

# Sub-agent 2: Pitch Generator
pitch_subagent = {
    "name": "pitch-generator",
    "description": "Creates pitch content for startup. Call AFTER user chooses an idea.",
    "system_prompt": """
    You are a Pitch Expert. Create pitches following the 4 PILLARS:

    1. EXTREME CLARITY - One sentence description, everyone understands
    2. OBVIOUS VALUE - "With it vs without it?" clearly answered
    3. LOW FRICTION - Can be demoed in 30 seconds
    4. SMART PACKAGING - Visual storytelling

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎯 ONE-LINER: [Max 10 words]

    😫 PROBLEM:
    [Describe the pain point in 2 sentences]

    ✨ SOLUTION:
    [Describe the solution in 2 sentences]

    🔑 KEY FEATURES:
    1. [Feature 1]
    2. [Feature 2]
    3. [Feature 3]

    📊 KEY METRIC: [1 specific number]

    🎬 CTA: [Call to action]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 3: Critic
critic_subagent = {
    "name": "critic",
    "description": "Evaluates and improves pitch quality. Call AFTER getting pitch content.",
    "system_prompt": """
    You are a strict Pitch Critic. Evaluate based on 4 criteria (1-5 points each):

    1. CLARITY (1-5): Does the listener understand in 10 seconds?
    2. VALUE (1-5): Is "with it vs without it" clear?
    3. UNIQUE (1-5): How is it different from existing solutions?
    4. VIBE (1-5): Would GenZ find it cool?

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 SCORE: X/20

    ✅ STRENGTHS:
    - [Point 1]
    - [Point 2]

    ⚠️ NEEDS IMPROVEMENT:
    - [Point 1]
    - [Point 2]

    💡 BETTER TAGLINE SUGGESTION: "..."

    🎯 VERDICT: [APPROVED ✓ if ≥16/20 | NEEDS IMPROVEMENT if <16/20]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    If score < 16: Explain SPECIFICALLY what needs improvement.
    """,
    "tools": []
}

# Sub-agent 4: Landing Page Content Builder
landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Creates AI Studio prompt for landing page. User copies this prompt into AI Studio Canvas mode to generate HTML. Call AFTER pitch is approved.",
    "system_prompt": """
    You are a Landing Page Content Specialist. Your task is to create a PROMPT + CONTENT
    for the user to copy into Google AI Studio Canvas mode to generate HTML.

    Based on the approved pitch content, create output in this format:

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📋 COPY INTO AI STUDIO CANVAS MODE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Create a complete HTML landing page for a startup with this information:

    ## STARTUP INFO
    - Name: [Startup name from pitch]
    - Tagline: [One-liner from pitch]

    ## HERO SECTION
    - Headline: [Compelling headline from pitch]
    - Subheadline: [Short description of the solution]
    - CTA Button: [CTA button text from pitch]

    ## PROBLEM → SOLUTION
    - Problem: [Pain point from pitch]
    - Solution: [Solution from pitch]

    ## 3 KEY FEATURES (with emoji icons)
    1. 🎯 [Feature 1]: [Short description]
    2. ⚡ [Feature 2]: [Short description]
    3. 🔒 [Feature 3]: [Short description]

    ## STYLE REQUIREMENTS
    - Color scheme: Modern gradient Purple (#7C3AED) → Green (#10B981)
    - Font: Inter (Google Fonts)
    - Layout: Mobile-first responsive
    - Effects: Smooth hover animations, subtle shadows
    - Sections: Hero, Problem/Solution, Features, CTA, Footer

    ## FOOTER
    - Copyright text with current year
    - Social media placeholder links (Facebook, Twitter, LinkedIn)

    Create complete HTML that can be immediately deployed to Vercel/Netlify.

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📌 HOW TO USE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    1. Open aistudio.google.com
    2. Select Canvas mode
    3. Paste the entire prompt above
    4. Click Generate
    5. Preview and download HTML
    6. Deploy to Vercel/Netlify
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 5: Social Post Writer
social_subagent = {
    "name": "social-writer",
    "description": "Creates Facebook post to share results. Call LAST after having landing page prompt.",
    "system_prompt": """
    Create a Facebook post for participants to share their results:

    REQUIREMENTS:
    - Tone: Excited but not cringe
    - Emoji: 3-5, placed appropriately
    - Max 280 characters
    - Required hashtags: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI

    TEMPLATE:
    🔥 [Hook - 1 attention-grabbing sentence]

    💡 Idea: [Startup name]
    🤖 [Number] AI agents working for me
    🌐 Live: [URL placeholder]

    [Hashtags]
    """,
    "tools": []
}

# ============================================
# CREATE STARTUP BUILDER ORCHESTRATOR
# ============================================
startup_builder = create_deep_agent(
    model=model,
    subagents=[
        personalization_subagent,
        pitch_subagent,
        critic_subagent,
        landing_page_subagent,
        social_subagent
    ],
    system_prompt="""
    You are the Startup Builder - an AI Orchestrator that helps users build startups from A to Z.

    ## WORKFLOW:

    **Step 1: Personalization**
    - If user provides profile → Call personalization to analyze and suggest ideas
    - If user already has an idea → Skip this step

    **Step 2: Pitch Generation**
    - Call pitch-generator to create pitch content

    **Step 3: Quality Control**
    - Call critic to evaluate the pitch
    - If score < 16/20 → Ask pitch-generator to improve and evaluate again
    - Loop until APPROVED (≥16/20)

    **Step 4: Landing Page Content**
    - Call landing-page-builder to create AI Studio prompt
    - Output is a PROMPT for user to copy into AI Studio, NOT HTML

    **Step 5: Social Post**
    - Call social-writer to create Facebook post

    ## NOTES:
    - Always follow the workflow in order
    - Critic MUST approve before moving to landing page step
    - Final output includes: Pitch + AI Studio Prompt + Facebook Post
    """
)

print("✅ Startup Builder ready!")
print("📋 Workflow: Profile → Ideas → Pitch → Critic → AI Studio Prompt → Social Post")

In [ ]:
# ============================================
# 🚀 RUN YOUR STARTUP BUILDER
# ============================================

# OPTION 1: With profile (to receive idea suggestions)
user_profile = {
    "name": "John Smith",
    "age": 22,
    "profession": "Software Developer",
    "interests": ["Platform Game", "Music", "AI"],
    "bio": "Innovative AI engineer who loves music and platform games"
}

result = startup_builder.invoke({"messages":
    [{"role": "user", "content": f"Come up with an idea based on my profile. Keep it concise. Profile: {user_profile}"}],
})

format_messages(result["messages"])

In [ ]:
print(result["messages"][-1].content)

In [ ]:
# ============================================
# OPTION 2: With existing idea (skip personalization)
# ============================================
chosen_idea = """*AI-Powered Music Platform Game:** Combining AI to create unique platform game levels based on music.
AI can analyze rhythm, melody and create matching challenges"""

result = startup_builder.invoke({"messages":
    [{"role": "user", "content": f"Build startup for idea: {chosen_idea}. Create pitch, get critic approval, AI Studio prompt, Facebook post. Keep it concise."}],
})

format_messages(result["messages"])

In [ ]:
print(result["messages"][-1].content)

---

## 🎁 BONUS: Hackathon Power Tools

**Now that you know how to build agents and set up a fullstack app, apply it to hackathons!**

### 🚀 Roadmap: MVP → Pitch

```
📋 MVP Planner  →  🎤 Pitch Generator
(Planning Agent)     (Pitch Agent)
      ↓                    ↓
 Project Brief       Landing Page
```

**Goal:** In 25 minutes, you will have:
- ✅ Detailed project brief for MVP
- ✅ Professional pitch deck/landing page

---

## 📋 Bonus 1: MVP Planning Agent (15 min)

### Goal
Transform an idea into a detailed **Project Brief** to build an MVP during a hackathon.

### Agent Workflow

```
💡 Hackathon Idea
     ↓
🎯 User Empathy Analysis
     ↓
✨ Feature Generation
     ↓
🔍 MVP Scope (Minimalism)
     ↓
📄 Project Brief → AI Studio Builder
```

### Why Do You Need a Planning Agent?

**Problem:** Great idea but:
- ❌ Scope too broad → can't finish during hackathon
- ❌ Not focused on core value
- ❌ Over-engineering

**Solution:** Planning Agent automatically:
- ✅ Analyzes user needs
- ✅ Generates features + prioritizes
- ✅ Shortlists MVP scope (enough to demo, not too complex)
- ✅ Creates clear brief for AI builders

In [ ]:
# ============================================
# 📋 Create MVP Planning Agent
# ============================================

mvp_planning_subagent = {
    "name": "mvp-planning",
    "description": "Analyzes hackathon ideas and creates detailed MVP project briefs.",
    "system_prompt": """
    You are an MVP Planning Expert for hackathons.

    TASKS:
    1. Analyze target users and pain points
    2. Generate features and prioritize using MOSCOW method
    3. Create MVP brief focused on core value

    MINIMALISM PRINCIPLES:
    - MVP = Minimum **Viable** Product (enough to validate, NOT complete)
    - Only keep MUST-HAVE features for demo
    - Don't over-engineer: Simple > Complex in hackathons
    - Focus: 3-5 core features is enough

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📋 MVP PROJECT BRIEF
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    ## 🎯 PROJECT OVERVIEW
    - Project Name: [Name]
    - One-liner: [1 sentence description]
    - Target Users: [Who will use it]

    ## 😫 PROBLEM STATEMENT
    [Main pain point in 2-3 sentences]

    ## ✨ SOLUTION
    [Solution in 2-3 sentences]

    ## 🔑 MUST-HAVE FEATURES (MVP)
    1. [Feature 1] - WHY: [Why it's necessary]
    2. [Feature 2] - WHY: [Why it's necessary]
    3. [Feature 3] - WHY: [Why it's necessary]

    ## ⏭️ NICE-TO-HAVE (Post-MVP)
    - [Features not needed for hackathon]

    ## 🛠️ SUGGESTED TECH STACK
    - Frontend: [Simple option]
    - Backend: [Simple option]
    - AI: Gemini API

    ## 📊 SUCCESS METRICS
    - [Metrics to measure success]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    📌 Copy this brief into AI Studio Builder or WebSim to generate your app!
    """,
    "tools": []
}

mvp_planning_agent = create_deep_agent(
    model=model,
    subagents=[mvp_planning_subagent],
    system_prompt="""
    You are the MVP Planning Orchestrator.
    When receiving a hackathon idea, call mvp-planning to create a detailed project brief.
    Output brief should be ready to copy into AI Studio Builder.
    """
)

print("✅ MVP Planning Agent ready!")

In [ ]:
# ============================================
# 🧪 Test MVP Planning Agent
# ============================================

test_idea = """
Hackathon idea: "Study Buddy AI"

Description: An AI app that helps students find suitable study partners based on:
- Subject/course
- Learning style (visual, auditory, kinesthetic)
- Schedule availability
- Goals (pass exam, deep understanding, project)

Uses Gemini AI for matching and suggesting study plans.
"""

result = mvp_planning_agent.invoke({"messages":
    [{"role": "user", "content": f"Please analyze this idea and create a detailed MVP brief:\n{test_idea}"}]
})

format_messages(result["messages"])

print("\n" + "="*50)
print("📋 Copy the brief above and paste into:")
print("   → https://aistudio.google.com/apps")
print("   → or https://websim.ai")

In [ ]:
print(result["messages"][-1].content)

### 🚀 Using the Brief with AI Builders

**Workflow:**
1. ✅ Copy MVP Brief from agent output
2. 🌐 Open [Google AI Studio Apps](https://aistudio.google.com/apps)
3. 📝 Paste brief into prompt
4. ⚡ AI Studio automatically generates:
   - React components
   - API endpoints
   - UI/UX design
   - Deployment config

**Alternative Builders:**
- [Builder.io](https://www.builder.io/) - Visual-first
- [Replit](https://replit.com/) - Full IDE
- [WebSim](https://websim.ai) - Instant prototypes

**💡 Pro Tip:** The more detailed your brief → The more accurate the generated app!

---

## 🎤 Bonus 2: Pitch Generator Agent (10 min)

### Goal
Create an **Irresistible Pitch** that turns judges from skeptical → excited!

### The 4 Pillars of Winning Pitches

```
🎯 Extreme Clarity  → Judges understand immediately what you do
💎 Obvious Value    → ROI/Impact is clear without explanation
⚡ Low Friction    → Smooth demo, easy to imagine adoption
📦 Smart Packaging  → Every slide has a job, no filler
```

### Pitch Structure

```
🎬 Hook (15s)         → "Imagine..."
💔 Problem (30s)      → Real pain + stats
✨ Solution (45s)     → Your product demo
📊 Traction (30s)     → Validation/MVP results  
🚀 Ask (15s)          → Clear next steps
```

**Total: 135 seconds = 2m15s** (perfect for 3-5 min pitch with Q&A buffer)

In [ ]:
# ============================================
# 🎤 Create Pitch Generator Agent
# ============================================

pitch_generator_subagent = {
    "name": "pitch-generator-bonus",
    "description": "Creates irresistible hackathon pitches with 4 pillars: clarity, value, low friction, smart packaging.",
    "system_prompt": """
    You are a Hackathon Pitch Expert - helping teams win competitions.

    4 PILLARS OF WINNING PITCHES:

    1️⃣ **EXTREME CLARITY**
       - Judges understand product in 10 seconds
       - No jargon, no complexity
       - Format: "[Product] helps [User] do [Action] in [Timeframe]"

    2️⃣ **OBVIOUS VALUE**
       - ROI is clear (time saved, money saved, problem solved)
       - Use numbers & comparisons
       - Before/After contrast

    3️⃣ **LOW FRICTION**
       - Demo must be smooth (no visible bugs!)
       - Judges can easily imagine adoption
       - Address objections proactively

    4️⃣ **SMART PACKAGING**
       - Every slide has a purpose
       - No filler content
       - Hook → Problem → Solution → Traction → Ask

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎤 HACKATHON PITCH DECK
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    ## 🎬 HOOK (15 seconds)
    "[Attention-grabbing opening statement]"

    ## 💔 PROBLEM (30 seconds)
    - Pain point 1
    - Pain point 2
    - Stats/Evidence

    ## ✨ SOLUTION (45 seconds)
    [Product Name]: [One-liner description]

    **Demo Flow:**
    1. Step 1
    2. Step 2
    3. Step 3

    ## 🔑 KEY FEATURES
    1. [Feature 1] → [Benefit]
    2. [Feature 2] → [Benefit]
    3. [Feature 3] → [Benefit]

    ## 📊 TRACTION/VALIDATION (30 seconds)
    - [Metric or user feedback]
    - [Technical achievement]

    ## 🚀 ASK (15 seconds)
    - [What you want from judges/investors]
    - CTA: [Clear next step]

    ## 💡 ONE-LINER TO REMEMBER
    "[Memorable tagline]"
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    📌 Copy this pitch into Gemini/Claude to generate landing page HTML!

    REMEMBER:
    - Judges see 20+ pitches → yours must POP
    - Story > Features
    - Passion > Polish (authenticity wins)
    """,
    "tools": []
}

pitch_generator_agent = create_deep_agent(
    model=model,
    subagents=[pitch_generator_subagent],
    system_prompt="""
    You are the Pitch Generator Orchestrator.
    When receiving hackathon project info, call pitch-generator-bonus to create a pitch deck.
    Output should be ready for presenter to use and copy into AI for landing page generation.
    """
)

print("✅ Pitch Generator Agent ready!")

In [ ]:
# ============================================
# 🧪 Test Pitch Generator Agent
# ============================================

pitch_request = """
Create a pitch deck for this hackathon idea:

**Study Buddy AI**
An AI app that matches students with suitable study partners based on:
- Learning style
- Subject/major
- Schedule
- Goals

Target: University students
Tech: Gemini AI + React + Firebase
MVP with working demo available

Need a 3-minute pitch to convince judges.
"""

result = pitch_generator_agent.invoke({"messages":
    [{"role": "user", "content": pitch_request}]
})

format_messages(result["messages"])

print("\n" + "="*50)
print("🎨 Next steps:")
print("   1. Copy pitch content above")
print("   2. Open Gemini or Claude")
print("   3. Prompt: 'Create an HTML landing page from this pitch:'")
print("   4. Paste pitch content")
print("   5. → Gemini generates beautiful landing page HTML!")
print("   6. Save as index.html and deploy to Vercel/Netlify")

In [ ]:
print(result["messages"][-1].content)

### 🎨 Using Pitch with Gemini Canvas

**Workflow: Pitch → Landing Page**

1. **Copy pitch output** from agent
2. **Open Gemini** (or Claude Canvas)
3. **Prompt:**
```
Create a modern, professional HTML landing page for this hackathon project.

Use these design principles:
- Hero section with compelling headline
- Problem-Solution sections with visuals
- Demo video/screenshots placeholder
- Clear CTA buttons
- Responsive design (Tailwind CSS)
- Smooth animations

Here's the pitch content:
[PASTE YOUR PITCH]
```

4. **Gemini generates** complete HTML + CSS
5. **Download** and deploy to:
   - [Vercel](https://vercel.com) (1-click)
   - [Netlify](https://netlify.com) (drag & drop)
   - [GitHub Pages](https://pages.github.com) (free)

**💡 Pro Tip:** Add demo video/GIF to landing page → conversion rate ↑↑↑

---

## 🎨 Deploy with AI Studio

**New workflow:** Agent output PROMPT → User copies into AI Studio → AI Studio generates HTML

### Steps:

1. **Copy prompt** from `landing-page-builder` agent output (the part in `📋 COPY INTO AI STUDIO`)
2. Open **[aistudio.google.com](https://aistudio.google.com)** → Select **Canvas mode**
3. **Paste** the entire copied prompt
4. Click **Generate** → AI Studio will create complete HTML
5. Click **Preview** → See landing page directly
6. **Download HTML** file
7. **Deploy:**
   - **Vercel:** [vercel.com/new](https://vercel.com/new) → Drop HTML file
   - **Netlify:** [app.netlify.com/drop](https://app.netlify.com/drop)

> 💡 **Tip:** The prompt is pre-formatted with all info from the pitch - just copy & paste!

---

## 📋 Idea Suggestions

| Category | Ideas |
|----------|--------|
| 🎮 Entertainment | Gaming guide, Meme generator, Playlist AI |
| 📚 Education | Study buddy, Homework helper, Language AI |
| 💼 Productivity | Meeting summarizer, Email writer, Task AI |
| 🛒 E-commerce | Price tracker, Review analyzer, Shopping AI |
| 🏥 Health | Fitness coach, Meal planner, Sleep AI |
| 💕 Social | Dating advisor, Event planner, Friend AI |
| 🎨 Creative | Story writer, Art director, Content AI |
| 🤪 Wildcard | Horoscope AI, Excuse generator, Roast bot |

---

## 📱 Facebook Share Template

```
🔥 Just built an AI Startup in 30 minutes at #DevFest!

💡 Idea: [STARTUP NAME]
🤖 Team of [NUMBER] AI agents working for me
🌐 Landing page LIVE: [LINK]

From idea → pitch → website in just 30 minutes.
This is the power of Multi-Agent AI! 🚀

#DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI #AIStartup #BuildInPublic
```

---

## 🔗 Resources

| Resource | Link |
|----------|------|
| DeepAgents Docs | https://docs.langchain.com/oss/python/deepagents |
| Gemini API | https://ai.google.dev |
| AI Studio | https://aistudio.google.com |
| Imagen API | https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview |
| Vercel Deploy | https://vercel.com/new |
| Netlify Drop | https://app.netlify.com/drop |

---

## 🎯 Key Takeaways



1. **Agent = AI that can think and plan**
2. **Tools = Give agents the ability to interact with the real world**
3. **Multi-Agent = Team of AIs working together**
4. **Critic = Quality control for output**

> 🔥 *"Startup founders validate ideas in 2 weeks. With AI Agents, you do it in 30 minutes."*

---


# 🙏 Thank You!

**Thank you for joining the workshop!**

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/You%20complete%20me.svg)

**Remember:** The best way to learn is to build. Go create something amazing! 🚀
**Congratulations! You've built your AI Startup Team!** 🎉